<a href="https://colab.research.google.com/github/data-analytics-arg/NewsScraper/blob/development/Scrapper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [11]:
# asignamos la web a scrapear a una variable, que llamamos url
url = 'https://www.pagina12.com.ar/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15'}

In [12]:
# hacemos un request de esa url y vemos estado
p12 = requests.get(url, headers=headers)
p12.status_code

200

In [13]:
p12.text

'<!DOCTYPE html><html class="no-js"><head><meta charset="utf-8"><title>Página12 | La otra mirada</title><meta name="google-site-verification" content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg"><script type="application/ld+json">{"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}</script><meta property="description" name="description" content="Notas focalizadas en el quehacer político de la Argentina."><meta property="fb:pages" name="fb:pages" content="1541638399393436"><meta property="og:locale" name="og:locale" content="es_AR"><meta property="og:title" name="og:title" content="Página12 | La otra mirada"><meta prope

In [14]:
p12.headers

{'Date': 'Tue, 05 Oct 2021 23:11:19 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'X-DNS-Prefetch-Control': 'off', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Download-Options': 'noopen', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'X-ETag': 'W/"66fc3-8bDUyuXLz68oP3l/dm4wxC2EaYo"', 'X-Backend': 'frontend', 'X-Type': 'Dynamic URI', 'X-Backend-TTL': '120.000', 'Age': '5', 'grace': '86400.000 none', 'ttl': '114.418', 'X-Instance': 'cache-front-prod-varnish-68cf7d4bd6-5lt52', 'x-restarts': '0', 'X-Cache': 'hit cached', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '699a54107ab61e89-AMS', 'Content-Encoding': 'gzip'}

In [15]:
p12.request.headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Safari/605.1.15', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [16]:
# parseamos el texto de la pagina principal
s = BeautifulSoup(p12.text, 'lxml')

In [20]:
print(s.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html class="no-js"><head><meta charset="utf-8"/><title>Página12 | La otra mirada</title><meta content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg" name="google-site-verification"/><script type="application/ld+json">{"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}</script><meta content="Notas focalizadas en el quehacer político de la Argentina." name="description" property="description"/><meta content="1541638399393436" name="fb:pages" property="fb:pages"/><meta content="es_AR" name="og:locale" property="og:locale"/><meta content="Página12 | La otra mirada" name="og:tit

In [25]:
secciones = s.find(id='main-nav-menu').find_all('a', attrs={'class': 'p12-dropdown-item'})



[<a class="p12-dropdown-item" href="https://www.pagina12.com.ar/secciones/el-pais">El país</a>,
 <a class="p12-dropdown-item" href="https://www.pagina12.com.ar/secciones/economia">Economía</a>,
 <a class="p12-dropdown-item" href="https://www.pagina12.com.ar/secciones/sociedad">Sociedad</a>,
 <a class="p12-dropdown-item" href="https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos">Espectáculos</a>,
 <a class="p12-dropdown-item" href="https://www.pagina12.com.ar/secciones/deportes">Deportes</a>,
 <a class="p12-dropdown-item" href="https://www.pagina12.com.ar/secciones/ciencia">Ciencia</a>,
 <a class="p12-dropdown-item" href="https://www.pagina12.com.ar/secciones/el-mundo">El mundo</a>,
 <a class="p12-dropdown-item" href="https://www.pagina12.com.ar/edicion-impresa">Edición impresa</a>,
 <a class="p12-dropdown-item" href="https://www.pagina12.com.ar/secciones/universidad-diario">Universidad</a>,
 <a class="p12-dropdown-item" href="https://www.pagina12.com.ar/secciones/ajedrez">Aj

In [29]:
# función para obtener las secciones en la pagina ppal del diario
def obtener_secciones(secciones):

        links_secciones = []
        for i in range(len(secciones)):
            links_secciones.append(secciones[i].get('href'))
        return links_secciones

In [31]:
links_secciones = [seccion.get('href') for seccion in secciones]
links_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/ciencia',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/edicion-impresa',
 'https://www.pagina12.com.ar/secciones/universidad-diario',
 'https://www.pagina12.com.ar/secciones/ajedrez',
 'https://www.pagina12.com.ar/secciones/cultura',
 'https://www.pagina12.com.ar/secciones/dialogos',
 'https://www.pagina12.com.ar/secciones/plastica',
 'https://www.pagina12.com.ar/secciones/psicologia',
 'https://www.pagina12.com.ar/secciones/cartas-de-lectores',
 'https://www.pagina12.com.ar/secciones/contratapa',
 'https://www.pagina12.com.ar/secciones/audiovisuales',
 'https://www.pagina12.com.ar/secciones/recordatorios',
 'https://www.pagina12.com.ar/secciones

In [36]:
# ahora repetimos el proceso, haciendo un request a cada elemento de la lista 'links_secciones':

request_de_seccion = requests.get(links_secciones[0], headers=headers)
request_de_seccion.status_code

200

In [40]:
soup_de_seccion = BeautifulSoup(request_de_seccion.text, 'lxml')
print(soup_de_seccion.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html amp="" lang="es"><head><meta charset="utf-8"/><title>El país | Página12</title><!-- DUST PATH: /usr/src/app/src/widgets/fc_jsonLD.dust/ --><script type="application/ld+json">{"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}</script><script type="application/ld+json">{"@context": "http://schema.org","@type": "NewsArticle","mainEntityOfPage": {"@type": "WebPage","@id": "https://www.pagina12.com.ar/secciones/el-pais"},"headline": "El país | Página12","isAccessibleForFree": "False","image": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_de

In [29]:
# creamos una funcion para que traiga todos los links de la seccion que le pongamos

def obtener_notas(soup_de_seccion):

    lista_notas = []

    # obtengo articulo principal
    noticia_h2 = soup_de_seccion.find('h2', attrs={'class': 'title-list'})
    if noticia_h2:
        lista_notas.append(noticia_h2.a.get('href'))
    # obtengo articulos restantes
    noticias_h3 = soup_de_seccion.find_all('h3', attrs={'class': 'title-list'})
    if noticias_h3:
        for i in range(len(noticias_h3)):
            lista_notas.append(noticias_h3[i].a.get('href'))

    noticias_h4 = soup_de_seccion.find_all('h4', attrs={'class': 'is-display-inline title-list'})
    if noticias_h4:
        for i in range(len(noticias_h4)):
            lista_notas.append(noticias_h4[i].a.get('href'))

    return (lista_notas)

In [30]:
lista_notas = obtener_notas(soup_de_seccion)

lista_notas_nueva=[]
for nota in lista_notas:
    nota.replace("''", "")
    nota.replace(" '", "")
    lista_notas_nueva.append(" http://www.pagina12.com.ar" + nota)



In [31]:
lista_notas_nueva

[]

In [32]:
lista_notas = lista_notas_nueva
lista_notas

[]

In [15]:
# hacemos el request para la nota
r_nota = requests.get(lista_notas_nueva[0])

IndexError: ignored

In [ ]:
# parseamos el contenido de esa pagina
s_nota = BeautifulSoup(r_nota.text, 'lxml')
print(s_nota.prettify())

<!DOCTYPE html>
<html class="no-js">
 <head>
  <meta charset="utf-8"/>
  <title>
   Pandora Papers: La trama offshore de la familia Macri | Cómo se conformó la red de empresas en paraísos fiscales | Página12
  </title>
  <link href="https://www.pagina12.com.ar/372643-pandora-papers-la-trama-offshore-de-la-familia-macri" rel="canonical"/>
  <link href="https://www.pagina12.com.ar/372643-pandora-papers-la-trama-offshore-de-la-familia-macri?amp=1" rel="amphtml"/>
  <script async="" src="https://news.google.com/swg/js/v1/swg.js">
  </script>
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagi

In [ ]:
# extraemos los distintos contenidos de la nota :

def obtener_info(s_nota):
    try:

        # creamos un diccionario vacio para ir metiendo la informacion
        ret_dict = {}



        # Extraemos la fecha
        fecha = s_nota.find('span', attrs={'pubdate': 'pubdate'})
        if fecha:
            ret_dict['fecha'] = fecha.get('datetime')
        else:
            ret_dict['fecha'] = None

        # Extraemos el titulo
        titulo = s_nota.find('h1', attrs={'class': 'article-title'})
        if titulo:
            ret_dict['titulo'] = titulo.get_text()
        else:
            ret_dict['titulo'] = None

        # Extraemos la volanta
        volanta = s_nota.find('h2', attrs={'class': 'article-prefix'})
        if volanta:
            ret_dict['volanta'] = volanta.get_text()
        else:
            ret_dict['volanta'] = None

        # Extraemos el copete

        copete = s_nota.find('div', attrs={'class': 'article-summary'})
        if copete:
            ret_dict['copete'] = copete.get_text()
        else:
            ret_dict['copete'] = None

        # Extraemos el cuerpo
        cuerpo = s_nota.find('div', attrs={'class': 'article-text'})
        if cuerpo:
            ret_dict['texto'] = cuerpo.get_text()
        else:
            ret_dict['texto'] = None

        # Extraemos el autor
        autor = s_nota.find('div', attrs={'class': 'article-author'})
        if autor:
            ret_dict['autor'] = autor.a.get_text()
        else:
            ret_dict['autor'] = None

        # extraemos imagen principal de la nota
        media = s_nota.find('div', attrs={'class': 'article-main-media-image'})
        if media:
            imagenes = media.find_all('img')
            if len(imagenes) == 0:
                print('no se encontraron imagenes')
            else:
                imagen = imagenes[-1]
                img_src = imagen.get('data-src')
                try:
                    img_req = requests.get(img_src)
                    if img_req.status_code == 200:
                        ret_dict['imagen'] = img_req.content
                    else:
                        ret_dict['imagen'] = None
                except:
                    print('No ha sido posible obtener la imagen')
        else:
            print('no se encontro media')
        # Extraemos la seccion
        seccion_nombre = s_nota.find('div', attrs={'class': 'suplement'})
        if seccion_nombre:
            ret_dict['seccion'] = seccion_nombre.a.get_text()
        else:
            ret_dict['seccion'] = None

        return ret_dict

    except Exception as e:
        print('Error')
        print(e)
        print('\n')

In [ ]:
obtener_info(s_nota)

no se encontro media


{'fecha': None,
 'titulo': None,
 'volanta': None,
 'copete': None,
 'texto': 'La investigación periodística global conocida como Pandora Papers arrojó nuevos datos sobre cuentas de argentinos en paraísos fiscales y expuso otra vez el patrimonio no declarado y la trama de empresas en paraísos fiscales de la familia Macri, al revelar la existencia de la sociedad offshore TNB Investments Limited, creada en Belice en 2016 por Mariano Macri, uno de los hermanos del expresidente Mauricio Macri. TNB Investments Limited no aparece registrada ante la Administración Federal de Ingresos Públicos (AFIP) y Mariano Macri fue su controlador, de acuerdo a la información obtenida en los archivos filtrados.Según una fuente que intervino en la conformación societaria, Mariano Macri quería abrir una cuenta bancaria para recibir 2,6 millones de dólares --un pago inicial de 350 mil dólares y treinta cuotas posteriores de 75 mil dólares-- como parte de un retiro de dividendos en su rol de accionista de Socm

In [ ]:
# definimos una funcion cuyo parametro va a ser la url de la nota que queremos scrapear
def scrape_nota(url_nota):
    try:
        r_nota = requests.get(url_nota)
    except Exception as e:
        print('Error scrapeando url', url_nota)
        print(e)
        return None
    if r_nota.status_code != 200:
        print(f'Error obteniendo nota {url_nota}')
        print(f'Status Code={r_nota.status_code}')
        return None
    s_nota = BeautifulSoup(r_nota.text, 'lxml')
    ret_dict = obtener_info(s_nota)
    ret_dict['url_nota'] = url_nota
    return ret_dict

In [ ]:
# armamos una lista con todas las notas que estan en cada una de las secciones (links_secciones)
notas = []
for i in range(len(links_secciones)):
    r = requests.get(links_secciones[i])
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'lxml')
        notas.extend(obtener_notas(soup))
    else:
        print('no se pudo obtener la seccion', link)

data = []

for i, nota in enumerate(notas):
    
    
    print(f'procesando nota: {i}/{len(notas)}')
    data.append(scrape_nota(" http://www.pagina12.com.ar" + nota))

procesando nota: 0/66
no se encontro media
procesando nota: 1/66
no se encontro media
procesando nota: 2/66
no se encontro media
procesando nota: 3/66
no se encontro media
procesando nota: 4/66
no se encontro media
procesando nota: 5/66
no se encontro media
procesando nota: 6/66
no se encontro media
procesando nota: 7/66
no se encontro media
procesando nota: 8/66
no se encontro media
procesando nota: 9/66
no se encontro media
procesando nota: 10/66
no se encontro media
procesando nota: 11/66
no se encontro media
procesando nota: 12/66
no se encontro media
procesando nota: 13/66
no se encontro media
procesando nota: 14/66
no se encontro media
procesando nota: 15/66
no se encontro media
procesando nota: 16/66
no se encontro media
procesando nota: 17/66
no se encontro media
procesando nota: 18/66
no se encontro media
procesando nota: 19/66
no se encontro media
procesando nota: 20/66
no se encontro media
procesando nota: 21/66
no se encontro media
procesando nota: 22/66
no se encontro medi

In [ ]:
# pasamos lo obtenido  a un df de pandas y exportamos a un .csv
df = pd.DataFrame(data)
#df.to_csv(f'{datetime.datetime.today().strftime("%d-%m-%Y-%H:%M:%S")}.csv')

In [ ]:
df

,fecha,titulo,volanta,copete,texto,autor,seccion,url_nota
0,None,None,None,None,Diego Santilli compartirá su primer acto de ca...,None,None,http://www.pagina12.com.ar/372652-diego-santi...
1,None,None,None,None,"Convertir los planes sociales en trabajo, el p...",None,None,http://www.pagina12.com.ar/372651-alberto-fer...
2,None,None,None,None,La investigación periodística global conocida ...,None,None,http://www.pagina12.com.ar/372643-pandora-pap...
3,None,None,None,None,El Tribunal Oral Federal de Mar del Plata cond...,None,None,http://www.pagina12.com.ar/372638-lesa-humani...
4,None,None,None,None,El presidente del Bloque del Frente de Todos e...,None,None,http://www.pagina12.com.ar/372611-maximo-kirc...
...,...,...,...,...,...,...,...,...
61,None,None,None,None,Desde Barcelona\nUNO Tres titulares de diario ...,None,None,http://www.pagina12.com.ar/371137-homo-normal...
62,None,None,None,None,"Bigotes en ristre, Salvador Dalí se confirma e...",None,None,http://www.pagina12.com.ar/370945-dali-en-con...
63,None,None,None,None,"Tristeza, amargura y un sentimiento de derrota...",None,None,http://www.pagina12.com.ar/370799-todos-nosotros
64,None,None,None,None,"La Patrulla Fronteriza de Texas, a caballo, ca...",None,None,http://www.pagina12.com.ar/370607-larga-vida-...
